# Imports

In [ ]:
import itertools as it
import os
import re
from collections import namedtuple
from functools import partial
from pathlib import Path

import dask
import distributed
import h5py
import holoviews as hv
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import skimage.measure
import zarr
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from holoviews.operation.datashader import regrid
from tqdm.auto import tqdm, trange

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from paulssonlab.image_analysis import *
from paulssonlab.image_analysis.ui import display_image

In [ ]:
%load_ext pyinstrument

In [ ]:
hv.extension("bokeh")

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="06:00:00",
    memory="2GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(1)

In [ ]:
cluster.adapt(maximum=20)

# Trench detection

In [ ]:
# filename = "/home/jqs1/scratch/jqs1/microscopy/210511/RBS_ramp.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230213/230213induction.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230215/230215induction.nd2" #v=7
# filename = "/home/jqs1/scratch/jqs1/microscopy/230326/230326promrbs.nd2" #v=8,t=10
filename = "/home/jqs1/scratch/jqs1/microscopy/230404/230404_rbsprom.nd2"

In [ ]:
nd2 = nd2reader.ND2Reader(filename)

# Drift correction test

In [ ]:
k1 = 8.947368421052635e-10
imgs = {
    t: image.correct_radial_distortion(nd2.get_frame_2D(v=8, c=0, t=t), k1=k1)[
        :500, :500
    ]
    for t in trange(225)
}

In [ ]:
img_x = imgs[20].copy()
y = 167
x = 476
img_x[y, x] = 20_000
img_x[y + 1, x] = 0
img_x[y - 1, x] = 0
img_x[y, x - 1] = 0
img_x[y, x + 1] = 0

In [ ]:
%%time
diag = util.tree()
trenches, info = trench_detection.find_trenches(
    imgs[20],
    # angle=np.deg2rad(89),
    # pitch=16.482897384305836,
    width_to_pitch_ratio=1.4 / 3.5,
    join_info=False,
    diagnostics=diag,
)

In [ ]:
info

In [ ]:
diag["labeling"]["set_finding"]["image_with_lines"]

In [ ]:
diag["labeling"]["set_finding"].keys()

In [ ]:
diag["labeling"]["set_finding"]["profiles"]

In [ ]:
diag["bboxes"]

In [ ]:
image_limits = geometry.get_image_limits(imgs[0].shape)

In [ ]:
rois = trenches  # [(trenches["trench_set"] == 1)][21:25]
t0 = 20
t1 = 225
ts = np.arange(t0 + 1, t1)
# ts = np.arange(200, t1)
# ts = [160,183]
# ts = [100]
shifts = {}
shifts[t0] = np.array([0, 0])
# shifts[t0] = np.array([0, 0])
features_plot = {}
rois_plot = {}
rois_final_plot = {}
for t in tqdm(ts):
    diag = {}
    # shift = drift.find_feature_drift(
    #     imgs[t - 1],
    #     imgs[t],
    #     trenches,
    #     initial_shift=shifts[t - 1],
    #     estimation_func=median_translation,
    #     max_iterations=3,
    #     diagnostics=diag,
    # )
    # if t > 180:
    #     f = drift.trench_cell_endpoints
    # else:
    #     f = mock_features
    shift = drift.find_feature_drift(
        imgs[t0],
        # imgs[t-1],
        imgs[t],
        rois,
        initial_shift2=shifts[t - 1],
        # feature_func=mock_features,
        # feature_func=drift.centroid,
        # estimation_func=drift.median_translation,
        max_iterations=2,
        diagnostics=diag,
    )
    shifts[t] = shift
    features_plot[t] = diag["features2"]
    rois_plot[t] = diag["rois2"]
    rois_final_plot[t] = diag["rois_final"]

In [ ]:
ts2 = ts  # [-30:]
hv.HoloMap({t: ui.RevImage(imgs[t]) for t in ts2}) * hv.HoloMap(
    {t: rois_final_plot[t] for t in ts2},
) * hv.HoloMap({t: features_plot[t] for t in ts2}).opts(color="red")

# Line drift correction

In [ ]:
roi = trenches.iloc[1]
top = (roi["top_x"], roi["top_y"])
bottom = (roi["bottom_x"], roi["bottom_y"])

In [ ]:
res = trench_detection.profile.profile_line(
    imgs[21], top[::-1], bottom[::-1], linewidth=10
)

In [ ]:
plt.imshow(res)

In [ ]:
plt.plot(res.mean(axis=0));

In [ ]:
def iter_roi_lines(rois):
    index = rois.index.values
    top_x = rois["top_x"].values
    top_y = rois["top_y"].values
    bottom_x = rois["bottom_x"].values
    bottom_y = rois["bottom_y"].values
    for i in range(len(index)):
        roi_idx = index[i]
        yield roi_idx, np.array([top_x[i], top_y[i]]), np.array(
            [bottom_x[i], bottom_y[i]]
        )


def line_shift(img, rois, linewidth=8):
    for roi_idx, top, bottom in iter_roi_lines(rois):
        res = trench_detection.profile.profile_line(
            img, top[::-1], bottom[::-1], linewidth=linewidth
        )
    pass


def find_line_drift(
    img1,
    img2,
    trenches,
    diagnostics=None,
):
    pass

In [ ]:
profiles = {}
for roi_idx, top, bottom in iter_roi_lines(rois):
    res = trench_detection.profile.profile_line(
        imgs[0], top[::-1], bottom[::-1], linewidth=10
    )
    profiles[roi_idx] = res

In [ ]:
profiles2 = {}
for roi_idx, top, bottom in iter_roi_lines(rois):
    res = trench_detection.profile.profile_line(
        imgs[40], top[::-1], bottom[::-1], linewidth=10
    )
    profiles2[roi_idx] = res

In [ ]:
idx = 20
plt.plot(profiles[idx].mean(axis=0))
plt.plot(profiles2[idx].mean(axis=0));

In [ ]:
plt.plot(np.mean(np.concatenate(list(profiles.values())), axis=0))

In [ ]:
for idx, res in profiles.items():
    horiz = res.mean(axis=0)
    horiz = horiz / horiz.max()
    plt.plot(horiz);

In [ ]:
for idx, res in profiles.items():
    vert = res.mean(axis=1)
    # vert = vert / vert.max()
    plt.plot(vert);